复现代码
2、比较使用高级API与使用低级API模型性能的差别，分析可能造成的原因

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

D:\AN\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
mnist = input_data.read_data_sets('MNIST_data',one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


使用便捷内置代价函数

In [5]:
with tf.Graph().as_default() as g:
    #输入、标记占位符
    inputs = tf.placeholder(shape=[None,784],dtype=tf.float32)
    labels = tf.placeholder(shape=[None,10],dtype=tf.float32)
    
    #创建128个隐藏层神经元参数
    hidden_weight = tf.Variable(tf.random_normal([784,128]),name='hidden_weight')
    hidden_bias = tf.Variable(tf.zeros([128, ]),name='hidden_bias')
    
    #隐藏层前向传播
    hidden_output = tf.nn.relu(tf.matmul(inputs,hidden_weight) + hidden_bias)
    
    #创建输出层10个神经元参数
    output_weight = tf.Variable(tf.random_normal([128,10],name='output_weight'))
    output_bias = tf.Variable(tf.zeros([10, ]),name='output_bias')
    
    #输出层前向传播
    logits = tf.matmul(hidden_output,output_weight) + output_bias
    output = tf.nn.softmax(logits)
    
    #代价函数
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=labels))
    
    #正确率
    acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(labels,axis=1),tf.argmax(output,axis=1)),tf.float32))
    
    
    with tf.Session(graph=g) as sess:
        #定义梯度下降法优化器
        optim = tf.train.GradientDescentOptimizer(learning_rate=0.01)
        train_op = optim.minimize(loss)
        sess.run(tf.global_variables_initializer())
        
        #训练模型
        for step in range(20000):
            batch_images,batch_labels = mnist.train.next_batch(32)
            res_loss,_ = sess.run([loss,train_op],feed_dict={inputs:batch_images,labels:batch_labels})
            
            #输出代价并验证模型：
            if step % 500 == 0:
                accs = []
                
                for test_step in range(10000 // 32):
                    batch_images,batch_labels = mnist.test.next_batch(32)
                    res_acc = sess.run(acc,feed_dict={inputs:batch_images,labels:batch_labels})
                    accs.append(res_acc)
                accs = np.mean(accs)
                print('step %5d,loss %2.4f,acc %.4f' % (step,res_loss,accs))


step     0,loss 73.5269,acc 0.0932
step   500,loss 5.1310,acc 0.7601
step  1000,loss 7.0535,acc 0.8116
step  1500,loss 5.1112,acc 0.8340
step  2000,loss 3.0746,acc 0.8465
step  2500,loss 4.0303,acc 0.8578
step  3000,loss 0.2184,acc 0.8671
step  3500,loss 0.2203,acc 0.8725
step  4000,loss 2.4274,acc 0.8718
step  4500,loss 2.7656,acc 0.8829
step  5000,loss 3.8996,acc 0.8789
step  5500,loss 0.0000,acc 0.8814
step  6000,loss 2.1060,acc 0.8867
step  6500,loss 0.9984,acc 0.8897
step  7000,loss 0.7719,acc 0.8885
step  7500,loss 1.8175,acc 0.8896
step  8000,loss 2.0435,acc 0.8951
step  8500,loss 1.0670,acc 0.8931
step  9000,loss 0.8269,acc 0.8942
step  9500,loss 0.9747,acc 0.8937
step 10000,loss 0.5493,acc 0.8935
step 10500,loss 0.0575,acc 0.8941
step 11000,loss 2.6666,acc 0.8952
step 11500,loss 1.4855,acc 0.8968
step 12000,loss 1.9012,acc 0.8966
step 12500,loss 1.4134,acc 0.9006
step 13000,loss 0.3089,acc 0.9001
step 13500,loss 1.7924,acc 0.8978
step 14000,loss 1.3304,acc 0.8995
step 14500,lo

In [ ]:
使用便捷的高级完成神经网络层的创建

In [4]:
with tf.Graph().as_default() as g:
    # 输入、标记占位符
    inputs = tf.placeholder(shape=[None, 784], dtype=tf.float32)
    labels = tf.placeholder(shape=[None, 10], dtype=tf.float32)
    
    # 隐藏层与输出层
    hidden_output = tf.keras.layers.Dense(
        128, activation=tf.nn.relu)(inputs)
    
    logits = tf.keras.layers.Dense(
        10, activation=None)(hidden_output)
    output = tf.nn.softmax(logits)
    
    # 代价函数
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
        logits=logits, labels=labels))
    
    # 正确率
    acc = tf.reduce_mean(
        tf.cast(tf.equal(tf.argmax(labels, axis=1), tf.argmax(output, axis=1)),
                tf.float32))


with tf.Session(graph=g) as sess:
    # 定义梯度下降法优化器
    optim = tf.train.GradientDescentOptimizer(learning_rate=0.01)
    train_op = optim.minimize(loss)
    sess.run(tf.global_variables_initializer())
    
    # 训练模型
    for step in range(20000):
        batch_images, batch_labels = mnist.train.next_batch(32)
        res_loss, _ = sess.run([loss, train_op], feed_dict={
            inputs: batch_images,
            labels: batch_labels
        })
        
        # 输出代价并验证模型
        if step % 500 == 0:
            accs = []
            for test_step in range(10000 // 32):
                batch_images, batch_labels = mnist.test.next_batch(32)
                res_acc = sess.run(acc, feed_dict={
                    inputs: batch_images,
                    labels: batch_labels
                })
                accs.append(res_acc)
            accs = np.mean(accs)
            print('step %5d, loss %2.4f, acc %.4f' % (step, res_loss, accs))

step     0, loss 2.3790, acc 0.1323
step   500, loss 0.5474, acc 0.8608
step  1000, loss 0.4044, acc 0.8880
step  1500, loss 0.4148, acc 0.8978
step  2000, loss 0.4638, acc 0.9051
step  2500, loss 0.2757, acc 0.9086
step  3000, loss 0.3705, acc 0.9106
step  3500, loss 0.2451, acc 0.9141
step  4000, loss 0.2879, acc 0.9171
step  4500, loss 0.1890, acc 0.9204
step  5000, loss 0.1310, acc 0.9251
step  5500, loss 0.2982, acc 0.9236
step  6000, loss 0.3924, acc 0.9258
step  6500, loss 0.2321, acc 0.9271
step  7000, loss 0.1846, acc 0.9293
step  7500, loss 0.3247, acc 0.9322
step  8000, loss 0.1611, acc 0.9334
step  8500, loss 0.0872, acc 0.9339
step  9000, loss 0.3946, acc 0.9342
step  9500, loss 0.1182, acc 0.9367
step 10000, loss 0.1133, acc 0.9373
step 10500, loss 0.1640, acc 0.9382
step 11000, loss 0.0629, acc 0.9395
step 11500, loss 0.0667, acc 0.9403
step 12000, loss 0.1430, acc 0.9411
step 12500, loss 0.1724, acc 0.9417
step 13000, loss 0.1606, acc 0.9435
step 13500, loss 0.1356, acc

高级API是为了降低大家的编码难度而设置的。Tensorflow更高层的API使得配置，训练，评估多种多样的机器学习模型更简单方便了。